# Libs

In [1]:
from bs4 import BeautifulSoup
from datetime import date
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
import requests

# Functions

In [2]:
def the_crawler(days_meal):
    url = "https://www.eatthismuch.com/diet-plan/3000-calorie/"
    chrome_options = Options()    
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
    driver.get(url)
    driver.find_element_by_css_selector(".btn.btn-lg.btn-block.btn-orange.gen_button").click()
    time.sleep(5)
    html_soup = BeautifulSoup(driver.page_source)
#     driver.close()
    return html_soup

def scrape_web(url):    
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"}
    ua=UserAgent()
    hdr = {"User-Agent": ua.random,
          "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
          "Accept-Charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.3",
          "Accept-Encoding": "none",
          "Accept-Language": "en-US,en;q=0.8",
          "Connection": "keep-alive"}
    response = requests.get(url, headers=hdr)
    time.sleep(5)
    html_soup = BeautifulSoup(response.content,"lxml")   
    return html_soup

# Meal Planner

In [3]:
meals = []

for days_meal in range(1):
    html_soup = the_crawler(days_meal)
    calories = html_soup.find_all("div", {"class": "plan_calories text-medium"})[0].text
    calories = calories.replace(" Calories","")

    for count in range(0,4): # 4 meals per day
        meal_plan = (html_soup.find_all("div", {"meal_box meal_container row"})[count])
        type_meal = meal_plan.find_all("div", {"col-auto text-dark-gray text-large text-strong print_meal_title wrap_or_truncate pr-0"})[0].text
        rece = meal_plan.find_all("div", {"print_name"})

        for i in range(len(rece)):
            meal = rece[i].text.strip()
            meal_link = meal.replace(" & ","%20")
            meal_link = meal_link.replace(" ","%20")
            meal_link = "https://www.eatthismuch.com/food/browse/?q="+meal_link
            day = days_meal
            meals.append([day,calories,type_meal, meal,meal_link])

    df = pd.DataFrame(meals)
    df.to_csv(f"../data/temp/receipts_partial.csv")



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/ericsonmarques/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/pl/wm7wjyvn1vvflk70yn1b6n6c0000gn/T/ipykernel_15945/3138219237.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


# Get link from recipe and nutrition values

## read files

In [4]:
# results from our crawler
df_now = pd.read_csv(f"../data/temp/receipts_partial.csv", index_col=[0])
df_now.columns = ["day", "cals", "type_meal", "meal", "link"]

# results from our previous crawler
df_previous = pd.read_csv(f"../data/processed/datafera_todos.csv", index_col=[0])

## Read links from recipe

In [5]:
links_recipes = set(df_now["link"]) - set(df_previous["link"])
links_recipes = list(links_recipes)

## Nutrition values
Get values from nutrition

In [6]:
dataframe_recipes = []
error=[]
receitas = []

for i in range(len(links_recipes)):
    
    try:
        
        url = links_recipes[i]
        html_soup = scrape_web(url)
        Title = html_soup.find_all("div", {"result_name col-3"})[0].text.strip()
        Calories = html_soup.find_all("div", {"col-2 offset-1 nutrient_cell"})[0].text.strip()
        Carbs = html_soup.find_all("div", {"col-2 nutrient_cell"})[0].text.strip()
        Fat = html_soup.find_all("div", {"col-2 nutrient_cell"})[1].text.strip()
        Protein = html_soup.find_all("div", {"col-2 nutrient_cell"})[2].text.strip()
        Fiber = html_soup.find_all("div", {"col-3 nutrient_cell"})[0].text.strip()
        receita = html_soup.find_all('a', {'class': 'row'})[0]["href"]
        receita = "https://www.eatthismuch.com"+receita
        dataframe_recipes.append([url,receita,Title,Calories,Carbs,Fat,Protein,Fiber])
        
    except:
        print(url)
        pass

## Cleaning Data

In [7]:
df_nutrition = pd.DataFrame(dataframe_recipes)
df_nutrition

,0,1,2,3,4,5,6,7
0,https://www.eatthismuch.com/food/browse/?q=Pas...,https://www.eatthismuch.com/recipe/nutrition/p...,Pasta la Checca,360,48.8\n ...,14.7\n ...,8.9\n ...,3.7\n ...
1,https://www.eatthismuch.com/food/browse/?q=Str...,https://www.eatthismuch.com/recipe/nutrition/p...,Peach and Strawberry Smoothie,128,30.2\n ...,0.4\n ...,2.0\n ...,5.0\n ...


In [8]:
df_nutrition = pd.DataFrame(dataframe_recipes)
df_nutrition.columns = ["link","receita","title","Calories","Carbs","Fat","Protein","Fiber"]
df_nutrition["Calories"] = df_nutrition["Calories"].str.replace("\D","",regex=True)
df_nutrition["Carbs"] = df_nutrition["Carbs"].str.replace("\D","",regex=True)
df_nutrition["Fat"] = df_nutrition["Fat"].str.replace("\D","",regex=True)
df_nutrition["Protein"] = df_nutrition["Protein"].str.replace("\D","",regex=True)
df_nutrition["Fiber"] = df_nutrition["Fiber"].str.replace("\D","",regex=True)

# Nutrition tables
the table df_now does not have info about the nutrition values, so we are going to add

In [9]:
# Get nutrition info cols
nutrition_info_cols = df_nutrition.columns

# from df_previous lets get only the nutrion information
df_nutrition_old = df_previous[nutrition_info_cols]

# add all nutrition info into a single table
df_now_nutrition = pd.concat([df_nutrition, df_nutrition_old])

# get single results
df_now_nutrition = df_now_nutrition.drop_duplicates()

In [10]:
df_now_nutrition = pd.merge(df_now, df_now_nutrition, on="link",how="left")

In [11]:
# get the last day value on previous dataframe, and add ascending days to the plan
df_now_nutrition["day"] = df_now_nutrition["day"]+(df_previous["day"].iloc[-1]+1)

In [12]:
# Adding previous results with the new ones
final_results = pd.concat([df_previous,df_now_nutrition])

In [13]:
# final_results.loc[final_results["type_meal"].str.contains("Breakfast", na=False), "obs"] = 1
# final_results.loc[final_results["type_meal"].str.contains("Lunch", na=False), "obs"] = 2
# final_results.loc[final_results["type_meal"].str.contains("Dinner", na=False), "obs"] = 2
# final_results.loc[final_results["type_meal"].str.contains("Snack", na=False), "obs"] = 1
# final_results["Calories"] = (final_results["Calories"]).astype(float)/(final_results["obs"]).astype(float)

In [14]:
final_results["Meal"] = "<a href=" + final_results["receita"]+">"+final_results["meal"]+"</a>"

In [15]:
final_results = final_results.drop(columns=["obs"])

In [16]:
final_results.to_csv("../app/food.csv")

In [17]:
sadasdasd

NameError: name 'sadasdasd' is not defined

# Get Ingredients

In [ ]:
ingredients = pd.read_csv("../data/processed/ingredients.csv", index_col=0)

In [ ]:
lista = []
links = list(set(ingredients["link_rece"])-set(final_results["receita"]))

if len(links) < 1:
    print(len(links), "nothing new")
    pass
    
else:
    for link in links:  
        try:
            html_soup = scrape_web(link)
            ingredientes = html_soup.find_all('span', {'itemprop': 'recipeIngredient'})
            comp_ingredientes = len(ingredientes)
            for i in range(comp_ingredientes):
                ingred = ingredientes[i].text.split("\n")
                ingred.append(link)
                lista.append(ingred)
            ingredients_new = pd.DataFrame(lista)
            ingredients_new.columns       = ["0","measure","amount","item","4","link_rece"]
            ingredients.columns = ["0","measure","amount","item","4","link_rece"]
            ingredients_new = pd.concat([ingredients_new,ingredients],0)
            ingredients_new.to_csv("../data/processed/ingredients.csv")
        except:
            print(link)
            pass

---

## libs

In [ ]:
import pandas as pd

# make market list

In [ ]:
final_results = pd.read_csv(f"../data/processed/datafera_todos.csv", index_col=[0])
ingredients = pd.read_csv("../data/processed/ingredients.csv", index_col=0)
ingredients["measure"] = ingredients["measure"].str.strip()
ingredients["amount"] = ingredients["amount"].str.strip()
ingredients["item"] = ingredients["item"].str.strip()

## measure

In [ ]:
ingredients.loc[ingredients["measure"]=="1 1/4", "measure"] = 1.25
ingredients.loc[ingredients["measure"]=="1 1/2", "measure"] = 1.50
ingredients.loc[ingredients["measure"]=="3 1/2", "measure"] = 3.50

In [ ]:
ingredients["measure2"] = ingredients.measure.apply(pd.eval)

## quantity

In [ ]:
def convertion(amount, ax):
    filter_ = ingredients["amount"].str.contains(amount)
    ingredients.loc[filter_, "measure3"] = ax * ingredients[filter_]["measure2"]
    return ingredients

## organizing

In [ ]:
ingredients = convertion("oz", 28.4)
ingredients = convertion("can", 160)
ingredients = convertion("cup", 200)
ingredients = convertion("tbsp", 15)
ingredients = convertion("tsp", 0.9)
ingredients = convertion("dash", 0.250)
ingredients = convertion("scoop", 30)
filter_ = (ingredients["item"]==("Chicken breast")) & (ingredients["amount"]!=("oz"))
ingredients.loc[filter_, "measure3"] = 236.0 * ingredients[filter_]["measure2"]

## fillna

In [ ]:
ingredients["measure4"] = ingredients["measure3"].fillna(ingredients["measure2"])

## final

In [ ]:
ingred = final_results[final_results["day"]<=30]["receita"].unique()
ingred = ingredients[ingredients["link_rece"].isin(ingred)]
output = ingred.groupby("item")[["measure4"]].sum().reset_index()
output.to_excel("output.xlsx")  

## output

In [ ]:
output.sample(10)

In [ ]:
output = ingred.groupby("item")[["measure4"]].sum().reset_index()
output['item'] = output['item'].str.lower()
common = pd.Series(' '.join(output['item']).split()).value_counts()[:]

In [ ]:
from newspaper import Article
import nltk
nltk.download('punkt')

url = "https://www.eatthismuch.com/recipe/nutrition/peanut-butter-protein-oats,905819/"
article = Article(url)

# Download html
article.download()

# Get information 
article.parse()
article.nlp()

In [ ]:
print("title",article.title, "\n")
print("publish_date",article.publish_date, "\n")
print("top_image",article.top_image, "\n")
print("summary",article.summary, "\n")
print("keywords",article.keywords, "\n")

In [ ]:
print(4067.0000/10639)
print(3450.0000/10639)
print(660.0000/10639)
print(1767.0000/10639)
print(695.0000/10639)

In [ ]:
# final_results.to_csv("../data/processed/datafera_todos.csv")